# Item Collaborative Filtering
* See `ItemCollaborativeFilteringBase.ipynb` for algorithm details
* We construct a graph $G$ where the vertices are items and edges are relations (sequel, prequel, etc.)
* $w_{ij}$ is the length of the shortest path between items $i,j$ in $G$

In [1]:
name = "ItemCFRelated";

In [2]:
using NBInclude
@nbinclude("ItemCollaborativeFilteringBase.ipynb");

In [3]:
using DataFrames

## Compute similarity matrices

In [4]:
function get_similarity_matrix(relationships)
    file = "../../data/processed_data/related_series.csv"
    df = DataFrame(CSV.File(file))
    df = filter(x -> x.relationship ∈ relationships, df)
    num_items = maximum(get_split("training").item)
    sparse(
        df.source .+ 1,
        df.target .+ 1,
        fill(1.0, length(df.source)),
        num_items,
        num_items,
    )
end;

In [5]:
function all_pairs_shortest_paths(S)
    # uses the optimization that all edges are weight 1
    dists = fill(Inf, size(S)...)
    for v = 1:size(S)[1]
        dists[v, v] = 0
    end
    T = I(size(S)[1])

    @showprogress for epoch = 1:size(S)[1]
        T = T * S
        change = false
        for (i, j, v) in zip(findnz(T)...)
            if dists[i, j] > epoch
                dists[i, j] = epoch
                change = true
            end
        end
        if !change
            break
        end
    end
    dists
end;

In [6]:
function get_similarity_matrix_outdir(relationships)
    # if the matrix is already stored on disk, return its filepath
    # otherwise, regenerate the matrix and store it to disk
    outdir = "$name/$(hash(relationships))"
    if ispath("../../data/alphas/$outdir")
        return outdir
    end

    @debug "generating similarity matrix for relationships $relationships"
    S = get_similarity_matrix(relationships)
    S = collect(1 ./ all_pairs_shortest_paths(S))
    S = convert.(Float32, S)        
    write_params(Dict("S" => S), outdir = outdir)
    outdir
end;

## Setup hyperparameters

In [7]:
all_relations = [
    "side_story",
    "summary",
    "parent_story",
    "sequel",
    "prequel",
    "character",
    "alternative_version",
    "other",
    "spin_off",
    "alternative_setting",
    "full_story",
];

In [8]:
@memoize function max_neighborhood_size(relation)
    # returns the size of the largest nonzero neighborhood
    S = read_params(get_similarity_matrix_outdir(relation))["S"]
    S = 1 ./ S
    maxK = 0
    for j = 1:size(S)[2]
        K = sum((S[:, j] .!= 0) .* (S[:, j] .!= Inf))
        if K > maxK
            maxK = K
        end
    end
    neighborhood_size = Int(round(maxK))
    @debug "using neighborhood size $neighborhood_size"
    neighborhood_size
end;

In [9]:
downcast_to_int(x) = isinteger(x) ? Int(x) : x
alphas = [
    ["UserItemBiases"]
    ["ItemCF.$K" for K in downcast_to_int.([2^4, 2^6, 2^8, 2^10])]
    ["ItemCFResid.$K" for K in downcast_to_int.([2^4, 2^6, 2^8, 2^10])]
    ["MatrixFactorization.$K" for K in downcast_to_int.([10, 20, 40])]
]
params = [
    cf_params(
        name = "ItemCFRelated.$name",
        training_residuals = alphas,
        validation_residuals = alphas,
        neighborhood_type = "abs",
        S = get_similarity_matrix_outdir(relation),
        K = max_neighborhood_size(relation),
        λ = [1.0, 1.0, 0.0],
    ) for (relation, name) in zip([all_relations], ["all"])
];

[ Debug: 20220102 15:32:07 using neighborhood size 361
[ Debug: 20220102 15:32:12 using neighborhood size 96


## Train models

In [ ]:
for param in params
    optimize_model(param)
end

Progress: 100%|███████████████████████████| Time: 0:04:02 ( 0.24  s/it)
[ Debug: 20220102 15:39:49 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2026091925091817,-0.0005046458762923906,0.0011950563147905774,-0.0030051603196605465) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(0.13348222833401327,0.0010533778584762557,-0.08238862115986983,0.2048051457670822): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, R

Iter     Function value   Gradient norm 
     0     1.202609e+00     3.005160e-03
 * Current step size: 1.0
 * time: 0.0275270938873291
 * g(x): [-0.0005046458762923906, 0.0011950563147905774, -0.0030051603196605465]
 * x: [1.0, 1.0, 0.0]


Progress: 100%|███████████████████████████| Time: 0:03:45 ( 0.23  s/it)
[ Debug: 20220102 15:44:08 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2025986936596715,-0.0005161890649113988,0.0011923077511231954,-0.0028654359438486546) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(0.1341906784777494,0.001460972965672447,-0.08255800603897992,0.20079243214846113): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, Ra

     1     1.201416e+00     1.267168e-03
 * Current step size: 371.66628930806434
 * time: 1533.3210401535034
 * g(x): [-0.0003063119460735653, -0.0012671675585218, -0.00042817648387216293]
 * x: [1.1875598602562092, 0.5558378539676161, 1.1169167847840717]


Progress: 100%|███████████████████████████| Time: 0:03:52 ( 0.23  s/it)
[ Debug: 20220102 16:09:46 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2012332855240604,-0.00019754096996008383,0.0009588272997030573,-0.00033892108834485785) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(0.3594277654707997,-7.609193956659956e-5,-0.15244860555326203,0.140493698596192): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, R

...

Progress: 100%|███████████████████████████| Time: 0:03:43 ( 0.23  s/it)
[ Debug: 20220102 18:47:51 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.200437146971658,2.4719825509992757e-8,2.0843087734176183e-7,-8.487981283743218e-9) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(2.7818971712161034,0.18940369987942243,-0.4663183747713371,0.12069178885908725): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, Ratings

    14     1.200437e+00     2.215917e-07
 * Current step size: 1.3564091778129723
 * time: 11804.85369515419
 * g(x): [-1.6581432234109618e-7, 2.215917109851512e-7, -4.930749360343894e-10]
 * x: [1.2938385347220331, 0.7716949998429187, 20.48133788634496]


Progress: 100%|███████████████████████████| Time: 0:03:37 ( 0.22  s/it)
[ Debug: 20220102 19:00:52 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2004371468562642,2.428621541294642e-8,-3.9108767086230235e-8,9.443039189903867e-11) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(2.7852844573800475,0.18975359768533542,-0.4667443692333104,0.12068680634054746): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, Rating

    15     1.200437e+00     4.239215e-09
 * Current step size: 0.8662440004264162
 * time: 12332.649432182312
 * g(x): [-1.1373674645480047e-9, -4.239214788102471e-9, 1.5883545709143802e-11]
 * x: [1.2939816407675164, 0.771667428264963, 20.484265894809955]


Progress:  14%|███▊                       |  ETA: 1:24:23 ( 5.54  s/it)